# Skipped Functions

# DataAggregator versions

In [ ]:
# edited version with omitted code chunks

# Use DataAggregator_alllocs function instead? 
# https://github.com/kamshee/CIS_PD/blob/master/DataPreprocessor_AllLocs2.ipynb
def DataAggregator(tasks,locs,clipsize=10000,overlap=0.9,startTS=0,endTS=1,append=1):#,symptom=None):
    """
    Agregate features (input) and metadata(output) for classification stage
    Apend is a flag on whether to create a new data matrix or aggregate new rows to existing one
    startTS and endTS specificies fraction of raw sensors data to use to generate clips

    OMIT: side specificies which score side to use (recommended to use the same side of the loca)
    MODIFY: symptom specifies which filter to use. None only applies a HP filter to the data
    """
    t0=time.time()

    #load subjects features data and assemble with subj metadata
    d = os.listdir(dict_path)
    fnames = [filename[0:4] for filename in d if filename.startswith('HC')]

    Data = pd.DataFrame() #the table with features and metadata
    
    #loop through subjects
    print(fnames)
    for subj_filename in fnames:

        #load current subject sensor data
        subj = int(subj_filename[2:4]) #subj numerical code
        f = open(os.path.join(dict_path,subj_filename),'rb')    
        act_dict = pickle.load(f)
        f.close()
        print('\nLoaded Subj %s sensor data'%subj)

        #loop through tasks for current subject
        for task in tasks: 
            print(task)

            #compute features from each sensor location and aggregate with metadata for current side           

            #loop through sensor locations and extract data - locations match the side
            for loc in locs:
                if loc in list(['medial_chest','sacrum']):
                    loc = loc
                else:
                    loc = loc+side                        
                print(loc)

                #high pass filter accelerometer data
                HPfilter(act_dict,task,loc)

                #generate clips and extract features for current task and sensor location
                clip_data = gen_clips(act_dict,task,loc,clipsize,overlap,False,startTS,endTS)
                feature_extraction(clip_data)
                #aggregate subject and features data
                n_rec = len(clip_data.keys()) #of sensor recordings
                print('Number of recordings = %d, location: %s'%(n_rec,loc))
                N = n_rec

                #aggregate data from each trial for current subject
                for i in range(N):
                    #features
                    # fix to avoid key error for certain sensors that ran out of battery, no data
                    if 'accel' in clip_data[i].keys():
                        if 'features' in clip_data[i]['accel'].keys():
                            D = clip_data[i]['accel']['features']
                            print('trial %d: %d clips generated'%(i,np.shape(D)[0]))
                            featcols = D.columns.tolist()

                            #metadata
# get trial value
                            D['Trial'] = #subj_score.visit[i]  # changed visit to trial
                            D['Task'] = task
                            D['Location'] = loc 
                            D['Subject'] = subj
                            Data = pd.concat([Data,D]) #concatenate data from each visit
                    else:
                        continue

    cols = ['Subject','Trial','Task','Location']+ featcols #, 'Visit','Bradykinesia','Tremor']+ featcols
    Data = Data[cols]    
    print('\nData matrix generated')
    print(Data.shape)

    saved_filename = 'DataFinal'+'.csv'#+symptom+'.csv'
    if append and os.path.isfile(os.path.join(features_path,saved_filename)):
        Data.to_csv(os.path.join(features_path,saved_filename),mode='a+',header=False) #append to existing
        print('Appending to existing Feature matrix ' + str(os.path.join(features_path,saved_filename)))    

    else:
        Data.to_csv(os.path.join(features_path,saved_filename))
        print('Feature matrix saved in ' + str(os.path.join(features_path,saved_filename)))
    
    t = time.time()
    eltime = (t-t0)/60
    print('Elapsed time = %.2f min'%eltime)

    #     return Data

In [ ]:
# original with ommited parts commented out
def DataAggregator(tasks,task_scores_all,locs,clipsize=10000,overlap=0.9,startTS=0,endTS=1,append=1):#,symptom=None):
    """
    Agregate features (input) and metadata(output) for classification stage
    Apend is a flag on whether to create a new data matrix or aggregate new rows to existing one
    startTS and endTS specificies fraction of raw sensors data to use to generate clips

    OMIT: side specificies which score side to use (recommended to use the same side of the loca)
    MODIFY: symptom specifies which filter to use. None only applies a HP filter to the data
    """
    t0=time.time()
#     #load subject scores (std motor assessments)
#     mot_scores = pd.read_excel(os.path.join(scores_path, 'Scores.xls'))
   
    #load subjects features data and assemble with subj metadata
    d = os.listdir(dict_path)
    fnames = [filename[0:4] for filename in d if filename.startswith('HC')]

    Data = pd.DataFrame() #the table with features and metadata
    
    #loop through subjects
    print(fnames)
    for subj_filename in fnames:

        #load current subject sensor data
        subj = int(subj_filename[2:4]) #subj numerical code
        f = open(os.path.join(dict_path,subj_filename),'rb')    
        act_dict_full = pickle.load(f) # change to act_dict since side doesn't matter
        f.close()
        print('\nLoaded Subj %s sensor data'%subj)

        T = tasks #make iterator
#         T = zip(tasks,task_scores_all) #make iterator
        
#         act_dict_left = gen_unimodal_data(act_dict_full, side='left')
#         act_dict_right = gen_unimodal_data(act_dict_full, side='right')

        #loop through tasks for current subject
        for task in T: 
#         for task,task_scores in T: 
            
            print(task)
#             ts = task_scores #copy variable

            #extract scores data for left and right side for current task
#             for side in ['left','right']:
                
#                 if side == 'left':
#                     act_dict = act_dict_left
#                 elif side == 'right':
#                     act_dict = act_dict_right
#                 else:
#                     act_dict = act_dict_full
                    
#                 #deal with exception for alternating and finger to nose (unilateral tasks)
#                 if (task_scores.find('alternating') > -1) | (task_scores.find('finger to nose') > -1):
#                     task_scores=ts+ ' ' + side
                
#                 print('Aggregating scores for %s side'%side)
            
#                 #extract scores for corresponding task, subject and side
#                 subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
#                                             task_scores+ ' ' + 'bradykinesia ' + side + ' upper limb',
#                                             task_scores+ ' ' + 'tremor ' + side + ' upper limb']]
#                 #change 1 month label to 4 wks for proper sorting
#                 mot_scores.loc[mot_scores['visit']=='1 Month','visit']='4 Weeks'
#                 #rename cols
#                 subj_score = subj_score.rename(index=str,
#                                                columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
#                 print(subj_score.head())

#                 subj_score.index = range(len(subj_score))
#                 if len(subj_score) < 1:
#                     print('no scores data for subject %d -- skipping..'%subj)

                #compute features from each sensor location and aggregate with metadata for current side
                else:             
                    
                    #loop through sensor locations and extract data - locations match the side
                    for loc in locs:
                        if loc in list(['medial_chest','sacrum']):
                            loc = loc
                        else:
                            loc = loc+side                        
                        print(loc)

                        #high pass filter accelerometer data
                        HPfilter(act_dict,task,loc)

                        #generate clips and extract features for current task and sensor location
# modify gen_clips
                        clip_data = gen_clips(act_dict,task,loc,clipsize,overlap,False,startTS,endTS)
                        feature_extraction(clip_data)
                        #aggregate subject, scores and features data
#                         n_visits = len(subj_score)    #of visits in Database
                        n_rec = len(clip_data.keys()) #of sensor recordings
                        print('Number of recordings = %d, location: %s'%(n_rec,loc))
#                         print('n_visits in score file = %d, # recordings = %d, location: %s'%(n_visits,n_rec,loc))
#                         N = n_visits
                        N = n_rec

#                         if n_visits != n_rec:
#                             print('# of recordings does not match # of visits! - matching first %d recordings'%(min([n_visits,n_rec])))
#                             N = min([n_visits,n_rec])

                        #aggregate data from each visit for current subject
                        for i in range(N):
                            #features
                            # fix to avoid key error for certain sensors that ran out of battery, no data
                            if 'accel' in clip_data[i].keys():
                                if 'features' in clip_data[i]['accel'].keys():
                                    D = clip_data[i]['accel']['features']
                                    print('trial %d: %d clips generated'%(i,np.shape(D)[0]))
                                    featcols = D.columns.tolist()
#                                     #scores
#                                     D['Bradykinesia'] = subj_score['Bradykinesia'][i]
#                                     D['Tremor'] = subj_score['Tremor'][i]
                                    #metadata
#                                     D['Visit'] = subj_score.visit[i] 
                                    D['Task'] = task
                                    D['Location'] = loc 
                                    D['Subject'] = subj
                                    Data = pd.concat([Data,D]) #concatenate data from each visit
                            else:
                                continue
                            
        # temp file saving in case of errors in later subjects disables ability to save entire csv
#         temp_filename = 'Data'+str(subj)+'.csv'
#         new_folder = os.path.join(features_path,'Temp')
#         Data.to_csv(os.path.join(new_folder,temp_filename))
#         print('Saved temp Feature matrix after including subj %d'%subj)

    cols = ['Subject','Task','Location']+ featcols #, 'Visit','Bradykinesia','Tremor']+ featcols
    Data = Data[cols]    
    print('\nData matrix generated')
    print(Data.shape)

    #save data to feature matrix
#     if symptom == None:
#         symptom=''
    saved_filename = 'DataFinal'+'.csv'#+symptom+'.csv'
    if append and os.path.isfile(os.path.join(features_path,saved_filename)):
        Data.to_csv(os.path.join(features_path,saved_filename),mode='a+',header=False) #append to existing
        print('Appending to existing Feature matrix ' + str(os.path.join(features_path,saved_filename)))    

    else:
        Data.to_csv(os.path.join(features_path,saved_filename))
        print('Feature matrix saved in ' + str(os.path.join(features_path,saved_filename)))
    
    t = time.time()
    eltime = (t-t0)/60
    print('Elapsed time = %.2f min'%eltime)

    #     return Data